# 2. 의사록 섹션 분리  

 - 괴상한 특수문자(.으로 잡지 못하는 특수문자)를 정규표현식을 이용하여 제거
 - 별도 제작한 정규표현식으로 섹션 분리

In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_excel('../data/금통위의사록(텍스트파일).xlsx', usecols = [1,2])
data.columns = ['date', 'text']
data.head(2)

,date,text
0,2021.7.15.,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,2021.7.29.,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [3]:
def change_by_re(x):
    x = re.sub('\n{2,}','', x)
    x = re.sub('- \d+ -', '', x)
    x = re.sub('―|｢|｣|-|[․/→←+]', '', x)
    return x
data['text'] = data['text'].apply(change_by_re)

In [4]:
data.head()

,date,text
0,2021.7.15.,2021년도 제14차금융통화위원회(정기) 의사록한 국 은 행1. 일 자 2021년 ...
1,2021.7.29.,2021년도 제15차금융통화위원회(정기) 의사록한 국 은 행1. 일 자 2021년 ...
2,2021.8.26.,2021년도 제17차금융통화위원회(정기) 의사록한 국 은 행1. 일 자 2021년 ...
3,2021.9.9.,2021년도 제18차금융통화위원회(정기) 의사록한 국 은 행1. 일 자 2021년 ...
4,2021.9.24.,2021년도 제19차금융통화위원회(정기) 의사록한 국 은 행1. 일 자 2021년 ...


### 문장으로 분리

In [5]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])+[ \s\n\.;\(]+|(?<=다)\.)\s+')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

### 섹션 분리

In [6]:
def txt_split(data):
    result_list = []
    for i in range(len(data)):
        time = data.loc[i, 'date']
        text = data.loc[i, 'text']

        p1 = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)|｢?.?외환.?국제금융\s?동향.?｣?과 관련하여.*' , text)
        p2 = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은|｢?.?금융시장\s?동향.?｣?과 관련하여,?', text)
        p3 = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', text)
        
        s1 = p1.start() if p1 else -1
        s2 = p2.start() if p2 else -1
        s3 = p3.start() if p3 else -1
        
        section_2 = text[s1:s2] if s1 >= 0 or s2 >= 0 else ''
        section_3 = text[s2:s3] if s2 >= 0 or s3 >= 0 else ''
        
        sentences2, text2 = tidy_sentences(section_2)
        sentences3, text3 = tidy_sentences(section_3)
        new_dict = {
            'date' : time,
            's2' : text2,
            's3' : text3
        }
        result_list.append(new_dict)
        
    df = pd.DataFrame(result_list)
    return df

In [7]:
data = txt_split(data)

In [8]:
data.head()

,date,s2,s3
0,2021.7.15.,(나) 외환‧국제금융 및 금융시장 동향일부 위원은 최근 미 장기금리가 하락세를 보이...,
1,2021.7.29.,,
2,2021.8.26.,(나) 외환‧국제금융 및 금융시장 동향일부 위원은 외환·금융시장에서는 델타 변이와 ...,
3,2021.9.9.,,
4,2021.9.24.,,


In [10]:
data.to_excel('../data/금통위의사록_섹션분리.xlsx')